In [95]:
import pkg_resources
from pkg_resources import DistributionNotFound, VersionConflict
from platform import python_version
import numpy as np
import pandas as pd
import time
import gc
import random
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import numpy as np 
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import seaborn as sns 
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from numpy import arange
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from yellowbrick.regressor import CooksDistance
from yellowbrick.datasets import load_concrete
from sklearn.metrics import mean_absolute_percentage_error
from datetime import date
import random
from sklearn.metrics import make_scorer
today = str(date.today())

In [96]:
random.seed(20)
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [99]:
# Bring in Experts vs Actuals file

nerds = pd.read_excel(f"/content/gdrive/MyDrive/DVA Project/Data/predictions_v_actuals/professional_projections_actuals_TE.xlsx")
espn = pd.read_excel(f"/content/gdrive/MyDrive/DVA Project/Data/predictions_v_actuals/ESPN300_actuals_TE.xlsx")
us = pd.read_excel(f"/content/gdrive/MyDrive/DVA Project/Data/predictions_v_actuals/Pred127_actuals_TE.xlsx")

In [100]:
espn['ESPN_Rank'] = espn['ESPN_pos_rank'].str[2:]

In [105]:
def compare_experts(pos):


  actuals = us[us['Pos_x'] == pos]
  actuals = actuals.sort_values('y',ascending=False)
  actuals['Rank_Actual'] = actuals.reset_index().index + 1
  actuals = actuals[['Player','Max_Team_Name','Pos_x','Rank_pred_pos','Rank_Actual']]
  actuals.columns = ['Player','Team','Pos','127_Rank','Rank_Actual']

  # Clean other dfs
  nerds_filtered = nerds[nerds['Position'] == pos]
  nerds_filtered = nerds_filtered[['Player','Team','Position','Pos_rank_prof_proj']]
  nerds_filtered.columns = ['Player','Team','Pos','Nerds_Rank']

  espn_filtered = espn[espn['pos'] == pos]
  espn_filtered = espn_filtered[['Player','Team','ESPN_Rank','pos']]
  espn_filtered.columns = ['Player','Team','ESPN_Rank','Pos']

  # Merge all together
  final = actuals.merge(nerds_filtered, left_on=['Player','Pos'], right_on=['Player','Pos'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
  final = final.merge(espn_filtered, left_on=['Player','Pos'], right_on=['Player','Pos'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
  final = final[['Player','Team','Pos','Rank_Actual','127_Rank','ESPN_Rank','Nerds_Rank']]

  # Calculate NDCG

  comparison = final.copy()

  comparison = comparison.sort_values('Rank_Actual')
  comparison['Weight'] = 0
  comparison['Weight'].iloc[0:5] = 1
  comparison['Weight'].iloc[5:10] = 2
  comparison['Weight'].iloc[10:15] = 4
  comparison['Weight'].iloc[15:20] = 8
  comparison['Weight'].iloc[20:] = 16

  #return comparison
  comparison = comparison.dropna()

  cols = ['127_Rank','ESPN_Rank','Nerds_Rank']
  comparison[cols] = comparison[cols].apply(pd.to_numeric, errors='coerce', axis=1)

  comparison['127_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['127_Rank']) / np.log10(x['Weight'] + 1), axis=1)
  comparison['ESPN_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['ESPN_Rank']) / np.log10(x['Weight'] + 1), axis=1)
  comparison['Nerds_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['Nerds_Rank']) / np.log10(x['Weight'] + 1), axis=1)

  # Calculate NDCG

  comparison_df = pd.DataFrame()

  top = comparison.iloc[0:12]
  top_127 = top['127_Error'].sum()
  top_ESPN = top['ESPN_Error'].sum()
  top_Nerds = top['Nerds_Error'].sum()

  bottom = comparison.iloc[12:24]
  bottom_127 = bottom['127_Error'].sum()
  bottom_ESPN = bottom['ESPN_Error'].sum()
  bottom_Nerds = bottom['Nerds_Error'].sum()

  top_df = pd.DataFrame([[pos,'Top 12',top_127, top_ESPN, top_Nerds]])
  bottom_df = pd.DataFrame([[pos,'Top 13:24',bottom_127, bottom_ESPN, bottom_Nerds]])

  comparison_df = pd.concat([comparison_df,top_df])
  comparison_df = pd.concat([comparison_df,bottom_df])
  comparison_df.columns=['Pos','Grouping','127 Error','ESPN Error','Nerds Error']


  #print(top)
  return comparison_df






In [106]:
compare_experts('QB')

,Player,Team,Pos,Rank_Actual,127_Rank,ESPN_Rank,Nerds_Rank,Weight
0,Josh Allen,Bills,QB,1,7.0,2,2.0,1
1,Tom Brady,Buccaneers,QB,2,8.0,9,9.0,1
2,Justin Herbert,Chargers,QB,3,9.0,8,8.0,1
3,Patrick Mahomes,Chiefs,QB,4,4.0,1,1.0,1
4,Matthew Stafford,Rams,QB,5,19.0,12,12.0,1
5,Aaron Rodgers,Packers,QB,6,2.0,6,7.0,2
6,Dak Prescott,Cowboys,QB,7,22.0,4,5.0,2
7,Joe Burrow,Bengals,QB,8,21.0,13,13.0,2
8,Kirk Cousins,Vikings,QB,9,10.0,22,19.0,2
9,Kyler Murray,Cardinals,QB,10,1.0,3,3.0,2


In [107]:
compare_experts('RB')

,Player,Team,Pos,Rank_Actual,127_Rank,ESPN_Rank,Nerds_Rank,Weight
0,Jonathan Taylor,Colts,RB,1,7.0,7,6.0,1
1,Austin Ekeler,Chargers,RB,2,10.0,8,14.0,1
2,Joe Mixon,Bengals,RB,3,4.0,13,12.0,1
3,James Conner,Cardinals,RB,4,11.0,33,35.0,1
4,Leonard Fournette,Buccaneers,RB,5,15.0,32,34.0,1
5,Ezekiel Elliott,Cowboys,RB,6,19.0,6,7.0,2
6,Alvin Kamara,Saints,RB,7,3.0,3,5.0,2
7,Antonio Gibson,Football Team,RB,8,9.0,11,11.0,2
8,Aaron Jones,Packers,RB,9,5.0,9,8.0,2
9,Josh Jacobs,Raiders,RB,10,18.0,20,19.0,2


In [108]:
compare_experts('WR')

,Player,Team,Pos,Rank_Actual,127_Rank,ESPN_Rank,Nerds_Rank,Weight
0,Cooper Kupp,Rams,WR,1,10.0,19,22.0,1
1,Davante Adams,Packers,WR,2,2.0,1,1.0,1
2,Deebo Samuel,49ers,WR,3,29.0,39,39.0,1
3,Justin Jefferson,Vikings,WR,4,7.0,8,6.0,1
4,Tyreek Hill,Chiefs,WR,5,4.0,2,2.0,1
5,Stefon Diggs,Bills,WR,6,1.0,3,8.0,2
6,Diontae Johnson,Steelers,WR,7,17.0,20,24.0,2
7,Mike Evans,Buccaneers,WR,8,5.0,14,9.0,2
8,Hunter Renfrow,Raiders,WR,9,46.0,92,88.0,2
9,Keenan Allen,Chargers,WR,10,13.0,10,15.0,2


In [ ]:
compare_experts('TE')

,Pos,Grouping,127 Error,ESPN Error,Nerds Error
0,TE,Top 12,126.954988,153.690319,146.125313
0,TE,Top 13:24,149.655905,109.955224,114.201060


In [ ]:
# Join Comparisons Together
qb = compare_experts('QB')
rb = compare_experts('RB')
wr = compare_experts('WR')
te = compare_experts('TE')

joined = pd.concat([qb, rb, wr, te])

joined.to_excel("/content/gdrive/MyDrive/DVA Project/Data/predictions_v_actuals/NDCG_comparisons.xlsx")

# Everything below is testing

In [ ]:
pos = 'TE'

actuals = us[us['Pos_x'] == pos]
actuals = actuals.sort_values('y',ascending=False)
actuals['Rank_Actual'] = actuals.reset_index().index + 1
actuals = actuals[['Player','Max_Team_Name','Pos_x','Rank_pred_pos','Rank_Actual']]
actuals.columns = ['Player','Team','Pos','127_Rank','Rank_Actual']

# Clean other dfs
nerds_filtered = nerds[nerds['Position'] == pos]
nerds_filtered = nerds_filtered[['Player','Team','Position','Pos_rank_prof_proj']]
nerds_filtered.columns = ['Player','Team','Pos','Nerds_Rank']

espn_filtered = espn[espn['pos'] == pos]
espn_filtered = espn_filtered[['Player','Team','ESPN_Rank','pos']]
espn_filtered.columns = ['Player','Team','ESPN_Rank','Pos']


espn_filtered.head()


,Player,Team,ESPN_Rank,Pos
21,Travis Kelce,KC,1,TE
32,Darren Waller,LV,2,TE
74,George Kittle,SF,3,TE
90,Mark Andrews,BAL,4,TE
91,T.J. Hockenson,DET,5,TE


In [ ]:
# Merge all together
final = actuals.merge(nerds_filtered, left_on=['Player','Pos'], right_on=['Player','Pos'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
final = final.merge(espn_filtered, left_on=['Player','Pos'], right_on=['Player','Pos'], how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')
final = final[['Player','Team','Pos','Rank_Actual','127_Rank','ESPN_Rank','Nerds_Rank']]


In [ ]:
# Calculate NDCG

comparison = final.copy()

comparison = comparison.sort_values('Rank_Actual')
comparison['Weight'] = 0
comparison['Weight'].iloc[0:5] = 1
comparison['Weight'].iloc[5:10] = 2
comparison['Weight'].iloc[10:15] = 4
comparison['Weight'].iloc[15:20] = 8
comparison['Weight'].iloc[20:] = 16

comparison = comparison.dropna()

comparison.head(20)

,Player,Team,Pos,Rank_Actual,127_Rank,ESPN_Rank,Nerds_Rank,Weight
0,Mark Andrews,Ravens,TE,1,2.0,4,4.0,1
1,Travis Kelce,Chiefs,TE,2,1.0,1,1.0,1
3,George Kittle,49ers,TE,4,8.0,3,3.0,1
4,Rob Gronkowski,Buccaneers,TE,5,4.0,16,12.0,1
5,Dallas Goedert,Eagles,TE,6,5.0,10,9.0,2
6,Mike Gesicki,Dolphins,TE,7,6.0,12,11.0,2
7,Dawson Knox,Bills,TE,8,27.0,26,27.0,2
8,Hunter Henry,Patriots,TE,9,16.0,15,16.0,2
9,Noah Fant,Broncos,TE,10,11.0,8,8.0,2
10,T.J. Hockenson,Lions,TE,11,7.0,5,6.0,4


In [ ]:
cols = ['127_Rank','ESPN_Rank','Nerds_Rank']
comparison[cols] = comparison[cols].apply(pd.to_numeric, errors='coerce', axis=1)

comparison['127_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['127_Rank']) / np.log10(x['Weight'] + 1), axis=1)
comparison['ESPN_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['ESPN_Rank']) / np.log10(x['Weight'] + 1), axis=1)
comparison['Nerds_Error'] = comparison.apply(lambda x: np.abs(x['Rank_Actual'] - x['Nerds_Rank']) / np.log10(x['Weight'] + 1), axis=1)



In [ ]:
# Calculate NDCG

comparison_df = pd.DataFrame()

top = comparison.iloc[0:12]
top_127 = top['127_Error'].sum()
top_ESPN = top['ESPN_Error'].sum()
top_Nerds = top['Nerds_Error'].sum()

bottom = comparison.iloc[12:24]
bottom_127 = bottom['127_Error'].sum()
bottom_ESPN = bottom['ESPN_Error'].sum()
bottom_Nerds = bottom['Nerds_Error'].sum()

top_df = pd.DataFrame([['TE','Top 12',top_127, top_ESPN, top_Nerds]])
bottom_df = pd.DataFrame([['TE','Top 13:24',bottom_127, bottom_ESPN, bottom_Nerds]])

comparison_df = pd.concat([comparison_df,top_df])
comparison_df = pd.concat([comparison_df,bottom_df])
comparison_df.columns=['Pos','Grouping','127 Error','ESPN Error','Nerds Error']

comparison_df



,Pos,Grouping,127 Error,ESPN Error,Nerds Error
0,TE,Top 12,126.954988,153.690319,146.125313
0,TE,Top 13:24,149.655905,109.955224,114.201060
